In [0]:
# Dim_Product1
spark.sql("""
CREATE OR REPLACE TABLE dim_product (
 ProductID INT,
 ProductName STRING,
 Category STRING
) USING DELTA
""")


# Dim_Customer
spark.sql("""
CREATE OR REPLACE TABLE dim_customer (
 CustomerID INT,
 CustomerName STRING,
 Region STRING
) USING DELTA
""")


# Dim_Date
spark.sql("""
CREATE OR REPLACE TABLE dim_date (
 DateID INT,
 FullDate DATE,
 Month STRING,
 Year INT
) USING DELTA
""")


# Fact_Sales
spark.sql("""
CREATE OR REPLACE TABLE fact_sales (
 OrderID INT,
 ProductID INT,
 CustomerID INT,
 DateID INT,
 Amount DOUBLE
) USING DELTA
""")


In [0]:
#Insert Data
spark.sql("""
INSERT INTO dim_product VALUES
(1, 'iPhone 14', 'Mobile'),
(2, 'MacBook Pro', 'Laptop')
""")


spark.sql("""
INSERT INTO dim_customer VALUES
(100, 'Alice', 'East'),
(101, 'Bob', 'West')
""")


spark.sql("""
INSERT INTO dim_date VALUES
(1, '2024-01-01', 'January', 2024),
(2, '2024-01-02', 'January', 2024)
""")


spark.sql("""
INSERT INTO fact_sales VALUES
(1000, 1, 100, 1, 799.00),
(1001, 2, 101, 2, 1299.00)
""")


In [0]:
# Star Schema Query
spark.sql("""
SELECT d.Region, p.Category, SUM(f.Amount) AS TotalSales
FROM fact_sales f
JOIN dim_customer d ON f.CustomerID = d.CustomerID
JOIN dim_product p ON f.ProductID = p.ProductID
GROUP BY d.Region, p.Category
""").show()


###Snowflake Schema

In [0]:
spark.sql("""
CREATE OR REPLACE TABLE dim_category (
 CategoryID INT,
 CategoryName STRING
) USING DELTA
""")


spark.sql("""
CREATE OR REPLACE TABLE dim_region (
 RegionID INT,
 RegionName STRING
) USING DELTA
""")


In [0]:
#normalized
spark.sql("""
CREATE OR REPLACE TABLE dim_product_sf (
 ProductID INT,
 ProductName STRING,
 CategoryID INT
) USING DELTA
""")


spark.sql("""
CREATE OR REPLACE TABLE dim_customer_sf (
 CustomerID INT,
 CustomerName STRING,
 RegionID INT
) USING DELTA
""")


In [0]:
spark.sql("""
INSERT INTO dim_category VALUES (1, 'Mobile'), (2, 'Laptop')
""")


spark.sql("""
INSERT INTO dim_region VALUES (1, 'East'), (2, 'West')
""")


spark.sql("""
INSERT INTO dim_product_sf VALUES (1, 'iPhone 14', 1), (2, 'MacBook Pro', 2)
""")


spark.sql("""
INSERT INTO dim_customer_sf VALUES (100, 'Alice', 1), (101, 'Bob', 2)
""")

In [0]:
spark.sql("""
SELECT r.RegionName, c.CategoryName, SUM(f.Amount) AS TotalSales
FROM fact_sales f
JOIN dim_customer_sf cu ON f.CustomerID = cu.CustomerID
JOIN dim_region r ON cu.RegionID = r.RegionID
JOIN dim_product_sf p ON f.ProductID = p.ProductID
JOIN dim_category c ON p.CategoryID = c.CategoryID
GROUP BY r.RegionName, c.CategoryName
""").show()


In [0]:
spark.sql("DROP TABLE IF EXISTS dim_product")
spark.sql("DROP TABLE IF EXISTS dim_customer")
spark.sql("DROP TABLE IF EXISTS dim_date")
spark.sql("DROP TABLE IF EXISTS fact_sales")
spark.sql("DROP TABLE IF EXISTS dim_category")
spark.sql("DROP TABLE IF EXISTS dim_region")
spark.sql("DROP TABLE IF EXISTS dim_product_sf")
spark.sql("DROP TABLE IF EXISTS dim_customer_sf")